# Getting Stock Data
First, lets import all the stuff we'll need to get pricing data

In [2]:
import numpy as np
import pandas as pd
import dill
import quandl
from sqlalchemy import create_engine, inspect
from tqdm import tqdm_notebook

Second things second, lets get our set of ticker symbols out of the pickle jar, and into a sorted list

In [2]:
tickSymbs = sorted(dill.load(open('setOfTickerSymbols.pkl', 'r')))

Now lets open my Quandl API key from its hidden and ignored file (so nobody scrapes it off this github repo)

In [3]:
quandl_key_handle = open("quandl.apikey", "r")
quandl.ApiConfig.api_key = quandl_key_handle.read()
quandl_key_handle.close()

#### Now lets download some pricing data!
...straight to a sqlite database now, no more dicts!

In [4]:
engine = create_engine('sqlite:///capstone.db')

In [5]:
dlFails = []
dlWins = []
for ticker in tqdm_notebook(tickSymbs):
    try:
        #tickerDF = quandl.get("WIKI/%s" % ticker)
        quandl.get("WIKI/%s" % ticker).to_sql(ticker, engine, if_exists = 'replace')
    except:
        dlFails.append(ticker)
    else:
        dlWins.append(ticker)

So I was able to download about 1/3 of my tickers of interest from Quandl. Lets make sure that the API request and `pandas.to_sql()` method actually did what I intended.

In [6]:
inspector = inspect(engine)
print len(inspector.get_table_names())
print inspector.get_table_names()

279
[u'AAAP', u'ABBV', u'ABIO', u'ACOR', u'ACRS', u'ACRX', u'ADHD', u'ADMP', u'ADMS', u'ADRO', u'ADXS', u'AERI', u'AEZS', u'AFMD', u'AGEN', u'AGIO', u'AGLE', u'AGN', u'AGRX', u'AGTC', u'AIMT', u'AKAO', u'AKBA', u'ALDR', u'ALDX', u'ALKS', u'ALNY', u'ALRN', u'ALXN', u'AMAG', u'AMGN', u'AMPH', u'AMRN', u'ANAB', u'ANIK', u'ANTH', u'APRI', u'AQXP', u'ARDM', u'ARDX', u'ARGS', u'ARGX', u'ARNA', u'ARQL', u'ARRY', u'AST', u'ATNM', u'ATNX', u'ATRS', u'AVEO', u'AVIR', u'AXON', u'AXSM', u'AZN', u'AZRX', u'BCRX', u'BHVN', u'BIIB', u'BLPH', u'BLRX', u'BLUE', u'BMRN', u'BMY', u'BOLD', u'BPTH', u'BTX', u'BYSI', u'CALA', u'CAPR', u'CATB', u'CBIO', u'CDTX', u'CELG', u'CEMP', u'CERC', u'CFRX', u'CHMA', u'CHRS', u'CLBS', u'CLDX', u'CLRB', u'CLSD', u'CLSN', u'CNAT', u'CNCE', u'CORI', u'CORT', u'CPRX', u'CRBP', u'CRMD', u'CTIC', u'CYCC', u'CYTK', u'CYTR', u'DARE', u'DBVT', u'DCTH', u'DERM', u'DMTX', u'DRRX', u'DVAX', u'EARS', u'EBIO', u'EGLT', u'EGRX', u'EIGR', u'EPZM', u'ESPR', u'EVOK', u'EXEL', u'EYEG', u

So now we have a sqlite DB with _some_ of the stock histories we wanted. Lets start feature extractions.
If needed, I'll buy premium DB access from Quandl and get the whole NYSE. 